In [142]:
load("../HLA_IBD_pairs_data.RData")
load("../GLMM_model_ready.RData")
##code for creating potential offspring genotypes
binding_subset <- age_gap_restrict
binding_subset <- binding_subset[, c(2,3,13)]
print(head(binding_subset))
binding_subset <- head(binding_subset)

# Updated child_genos function to return a one-row data frame
child_genos <- function(pair, HLA_sheet, locus) {
    female <- as.integer(strsplit(pair, split=":")[[1]][1])
    male <- as.integer(strsplit(pair, split=":")[[1]][2])
    HLA_sheet <- as.data.frame(HLA_sheet)
    full_df <- rbind(HLA_sheet[HLA_sheet$Sample == female, ], HLA_sheet[HLA_sheet$Sample == male, ])
    df <- full_df[, grepl(locus, colnames(full_df))]
    df <- as.matrix(df)
    
    genos <- c(df[1, 1], df[2, 1], df[1, 1], df[2, 2], df[1, 2], df[2, 1], df[1, 2], df[2, 2])
    
    if (locus == "HLA.DRB1") {
        genos <- sapply(genos, function(geno) {
            geno <- gsub("P$", "", geno)
            geno <- gsub(":", "", geno)
            geno <- gsub("DRB1", "DRB1_", geno)
            paste0(geno, "P")
        })
    }
    
    # Return a one-row data frame with the list of genotypes
    return(data.frame(genotypes = I(list(unname(genos))), stringsAsFactors = FALSE))
}

# Updated child_genos_class2 function to return a one-row data frame
child_genos_class2 <- function(pair, HLA_sheet, locus) {
    female <- as.integer(strsplit(pair, split=":")[[1]][1])
    male <- as.integer(strsplit(pair, split=":")[[1]][2])
    HLA_sheet <- as.data.frame(HLA_sheet)
    full_df <- rbind(HLA_sheet[HLA_sheet$Sample == female, ], HLA_sheet[HLA_sheet$Sample == male, ])
    
    df <- if (locus == "DP") full_df[, 9:12] else full_df[, 13:16]
    df <- as.data.frame(lapply(df, function(x) gsub("[:]", "", x)))
    df <- as.data.frame(lapply(df, function(x) gsub("P$", "", x)))
    df <- as.matrix(df)
    
    mom_gene1 <- df[1, 1:2]
    mom_gene2 <- df[1, 3:4]
    dad_gene1 <- df[2, 1:2]
    dad_gene2 <- df[2, 3:4]
    
    gene1_combos <- rbind(
        c(mom_gene1[1], dad_gene1[1]), c(mom_gene1[1], dad_gene1[2]),
        c(mom_gene1[2], dad_gene1[1]), c(mom_gene1[2], dad_gene1[2])
    )
    gene2_combos <- rbind(
        c(mom_gene2[1], dad_gene2[1]), c(mom_gene2[1], dad_gene2[2]),
        c(mom_gene2[2], dad_gene2[1]), c(mom_gene2[2], dad_gene2[2])
    )
    
    child_combinations_list <- list()
    for (i in 1:nrow(gene1_combos)) {
        for (j in 1:nrow(gene2_combos)) {
            gene1_combo <- gene1_combos[i, ]
            gene2_combo <- gene2_combos[j, ]
            new_combo <- c()
            for (g1_allele in gene1_combo) {           
                for (g2_allele in gene2_combo) {
                    combo <- paste0("HLA-", g1_allele, "-", g2_allele, "P")
                    new_combo <- c(new_combo, combo)
                }
            }
            child_combinations_list <- c(child_combinations_list, unname(new_combo))
        }
    }
    
    # Return a one-row data frame with the list of combinations
    return(data.frame(combinations = I(list(unlist(child_combinations_list))), stringsAsFactors = FALSE))
}

# A tibble: 6 x 3
  actorID partnerID pairID 
  <chr>   <chr>     <chr>  
1 770     75        770:75 
2 3045    75        3045:75
3 3111    75        3111:75
4 1078    75        1078:75
5 3109    75        3109:75
6 3054    75        3054:75


In [144]:
library(tibble) 

# Create lists of child genotypes for each locus
A <- do.call(rbind, lapply(1:nrow(binding_subset), function(i) {
  child_genos(pair = as.character(binding_subset[i, 3]), HLA_sheet = Final_sheet, locus = "HLA.A")
}))

B <- do.call(rbind, lapply(1:nrow(binding_subset), function(i) {
  child_genos(pair = as.character(binding_subset[i, 3]), HLA_sheet = Final_sheet, locus = "HLA.B")
}))

C <- do.call(rbind, lapply(1:nrow(binding_subset), function(i) {
  child_genos(pair = as.character(binding_subset[i, 3]), HLA_sheet = Final_sheet, locus = "HLA.C")
}))

# For binding_class2
DP <- do.call(rbind, lapply(1:nrow(binding_subset), function(i) {
  child_genos_class2(pair = as.character(binding_subset[i, 3]), HLA_sheet = Final_sheet, locus = "DP")
}))

DQ <- do.call(rbind, lapply(1:nrow(binding_subset), function(i) {
  child_genos_class2(pair = as.character(binding_subset[i, 3]), HLA_sheet = Final_sheet, locus = "DQ")
}))

DRB1 <- do.call(rbind, lapply(1:nrow(binding_subset), function(i) {
  child_genos(pair = as.character(binding_subset[i, 3]), HLA_sheet = Final_sheet, locus = "HLA.DRB1")
}))

# Combine the new data frames into binding_subset
binding_subset <- cbind(binding_subset, A, B, C, DP, DQ, DRB1)




save(binding_subset, file="child_genos.RData")                               


In [5]:
rm(list=ls())
load("class1_binding.RData")
ls()

[1] "binding_subset"

In [6]:
head(binding_subset)

,actorID,partnerID,pairID,genotypes,genotypes.1,genotypes.2,all_SB_A,all_SB_B,all_SB_C,all_WSB_A,⋯,ex_SB_C,ex_WSB_A,ex_WSB_B,ex_WSB_C,int_ex_SB_A,int_ex_SB_B,int_ex_SB_C,int_ex_WSB_A,int_ex_WSB_B,int_ex_WSB_C
,<chr>,<chr>,<chr>,<I<list>>,<I<list>>,<I<list>>,<list>,<list>,<list>,<list>,⋯,<list>,<list>,<list>,<list>,<list>,<list>,<list>,<list>,<list>,<list>
1,2977,3031,2977:3031,"A03:01P,....","B15:10P,....","C03:04P,....","3837, 3101, 3729, 3389","4278, 3944, 4310, 3059","4748, 5984, 5396, 4013","12122, 9484, 10911, 10459",⋯,"543, 609, 623, 442","1607, 1336, 1419, 1390","1277, 1512, 1429, 1314","1602, 1842, 1839, 1402","1335, 1056, 1298, 1143","1001, 1238, 1153, 1089","1277, 1663, 1546, 1184","3970, 3105, 3476, 3240","3590, 4425, 4029, 3825","3827, 4657, 4608, 3676"
2,134,3031,134:3031,"A03:01P,....","B45:01P,....","C07:01P,....","3837, 3101, 2994, 2434","4033, 3258, 3096, 3233","5396, 4013, 4151, 5791","12122, 9484, 9170, 7142",⋯,"623, 442, 503, 649","1607, 1336, 1225, 1003","1138, 1144, 1001, 1314","1839, 1402, 1495, 1858","1335, 1056, 1036, 827","987, 1108, 608, 963","1546, 1184, 1153, 1648","3970, 3105, 2913, 2254","3300, 3369, 2434, 3544","4608, 3676, 3500, 4538"
3,2985,3031,2985:3031,"A23:01P,....","B15:10P,....","C03:04P,....","3729, 3389, 2994, 2434","4278, 3944, 3096, 3233","4748, 5984, 5550, 4564","10911, 10459, 9170, 7142",⋯,"543, 609, 653, 490","1419, 1390, 1225, 1003","1277, 1512, 1001, 1314","1602, 1842, 1878, 1576","1298, 1143, 1036, 827","1001, 1238, 608, 963","1277, 1663, 1586, 1347","3476, 3240, 2913, 2254","3590, 4425, 2434, 3544","3827, 4657, 4742, 4150"
4,610,3031,610:3031,"A29:02P,....","B15:03P,....","C02:02P,....","3269, 2584, 2994, 2434","4402, 3769, 4278, 3944","3177, 5844, 4748, 5984","9622, 7544, 9170, 7142",⋯,"412, 697, 543, 609","1275, 1025, 1225, 1003","1509, 1556, 1277, 1512","1283, 1917, 1602, 1842","1138, 890, 1036, 827","1136, 1289, 1001, 1238","916, 1738, 1277, 1663","3104, 2410, 2913, 2254","3859, 4195, 3590, 4425","3001, 4727, 3827, 4657"
5,3013,3031,3013:3031,"A29:02P,....","B14:01P,....","C08:02P,....","3269, 2584, 2994, 2434","4586, 4414, 4033, 3258","4755, 4928, 4151, 5791","9622, 7544, 9170, 7142",⋯,"606, 563, 503, 649","1275, 1025, 1225, 1003","1327, 1638, 1138, 1144","1729, 1688, 1495, 1858","1138, 890, 1036, 827","1104, 1377, 987, 1108","1398, 1502, 1153, 1648","3104, 2410, 2913, 2254","3764, 4870, 3300, 3369","4178, 4426, 3500, 4538"
6,3131,3031,3131:3031,"A03:01P,....","B53:01P,....","C07:01P,....","3837, 3101, 3729, 3389","3096, 3233, 2405, 3507","5396, 4013, 5239, 4602","12122, 9484, 10911, 10459",⋯,"623, 442, 638, 514","1607, 1336, 1419, 1390","1001, 1314, 761, 1383","1839, 1402, 1949, 1570","1335, 1056, 1298, 1143","608, 963, 431, 947","1546, 1184, 1572, 1418","3970, 3105, 3476, 3240","2434, 3544, 1977, 3893","4608, 3676, 4734, 4103"
